# Introduction to Symata

Load Symata like this

In [1]:
using Symata;

Symata is a computer language written in Julia. After typing `using Symata`, we are still in Julia mode, and expressions are interpreted as Julia code

In [2]:
length(zeros(10)) ==  10

true

In `IJulia` (Jupyter), type `isymata()` to enter `Symata` mode. At the command line REPL, type `=` at the beginning of a line to enter `Symata` mode.

In [3]:
isymata()

**Note**: to leave `Symata` mode and return to `Julia` mode, type `Julia()` in `IJulia`, or backspace at the command line REPL.

Now we can enter `Symata` expressions.

## Entering expressions

You enter expressions and Symata evaluates them

In [4]:
Cos(π * x)

L"$$ \text{Cos} \!  \left(  \pi  \ x \right)  $$"

In [5]:
x = 1/3

L"$$ \frac{1}{3} $$"

In [6]:
Out(4)

L"$$ \frac{1}{2} $$"

In [7]:
x = 1/6

L"$$ \frac{1}{6} $$"

In [8]:
Out(4)

L"$$ \frac{3^{\frac{1}{2}}}{2} $$"

In [9]:
Clear(x)

In [10]:
Out(4)

L"$$ \text{Cos} \!  \left(  \pi  \ x \right)  $$"

It is clear what happened above. If `x` is not set to a value, then `Cos(π * x)` can't be written in a simpler form. If we set `x` to some particular values, then `Cos(π * x)` *can* be reduced to a simpler form.

*(you can skip the following the first time through)*

- But, the reason `Symata` understands this is a consquence of the procedure it follows in evaluating (almost) all expressions. `Symata` evaluates expressions to a fixed point. More precisely, when an expression is given as input, `Symata` descends depth-first evaluating each subexpression to a fixed point and finally the top-level expression to a fixed point. When `Cos(π * x)` is first evaluated, each of `π` and `x` evaluates to itself so that `π * x` is already at a fixed point. Since there is no rule for evaluating `Cos(π * x)` for fixed `π * x`, `Cos(π * x)` is also at a fixed point.

- The expression `x=1/3` means that, whenever `x` is encountered, it should evaluate to `1/3`. The expression `Out(4)` evaluates to the fourth output cell, which is `Cos(π * x)`. Then `π` evaluates to iteself, `x` evaluates to `1`, so that `π * x` evaluates to `π/3`. There is a rule saying that `Cos(x/3)` evaluates to `1/2`.

- `Clear(x)` says that `x` should once again evaluate to itself. Then evaluating `Out(4)` follows the same evaluation sequence, leading to `Cos(π * x)`

## Assigning values

There are several kinds of assignment in Symata. The two most common are  `=` (or `Set`) and `:=` (or `SetDelayed`).
`Set` immediatley evaluates the right hand side and binds the left hand side to the result. `SetDelayed` does not evaluate the right hand side when the assignment is made. It evaluates the right hand side every time the left hand side is subsequently evalutated and then binds the result.

The following demonstrates the difference.

In [11]:
(
  x = 1,
  a := x,
  b = x,
  c = a,
  d := a
  )

In [12]:
[x,a,b,c,d]

L"$$ [1,1,1,1,1] $$"

In [13]:
ClearAll(x)

In [14]:
[x,a,b,c,d]

L"$$ [x,x,1,1,x] $$"

In [15]:
(a = z, [x,a,b,c,d])

L"$$ [x,z,1,1,z] $$"

In [16]:
ClearAll(x,a,b,c,d)

## Expressions and parts of expressions

*Expression* is the central concept in Symata. In general, expressions are trees whose branches and leaves are other expressions. You can manipulate these expressions.

Every expression has a `Head`. For function-like expressions, the `Head` is the function name. For atomic expressions, the `Head` usually is a data type.

In [17]:
Map(Head, [x, x + y, [x,y], Cos(x), f(x), 3, 3.0, BI(3), BF(3)])  # apply the fun

L"$$ [\text{Symbol},\text{+},\text{List},\text{Cos},f,\text{Int64},\text{Float64},\text{BigInt},\text{BigFloat}] $$"

In [18]:
expr = Expand((x+y)^3)

L"$$ x^{3} + 3 \ x^{2} \ y + 3 \ x \ y^{2} + y^{3} $$"

In [19]:
FullForm(expr)   # This shows the internal form. The tree is explicit

Plus(Power(x,3),Times(3,Power(x,2),y),Times(3,x,Power(y,2)),Power(y,3))

In [20]:
expr[2,2,1]   # Return a part of the expression by index into the tree

L"$$ x $$"

In [21]:
expr[2,2,1] = z;  # Replace a part of the expression

L"$$ z $$"

In [22]:
expr

L"$$ x^{3} + 3 \ z^{2} \ y + 3 \ x \ y^{2} + y^{3} $$"

In [23]:
Part(expr,2,2,1)  # You can do the same thing with Part

L"$$ z $$"

In [24]:
Expand((x+y)^3)[4,1]   # You can get parts of expressions directly

L"$$ y $$"

In [25]:
expr = Expand((x+y)^20);

In [26]:
expr[14:18:2]  # Parts 14 through 18 with step 2

L"$$ 77520 \ x^{7} \ y^{13} + 15504 \ x^{5} \ y^{15} + 1140 \ x^{3} \ y^{17} $$"

In [27]:
ClearAll(expr)

## Definition

`Definition` shows the definitions associated with a symbol

In [28]:
ClearAll(x,a,b,c,d)  # delete definitions from the previous example

In [29]:
a = 1

L"$$ 1 $$"

In [30]:
Definition(a)

a=1



In [31]:
a := x

In [32]:
Definition(a)   # This overwrites the previous definition

a:=x



In [33]:
( f(x_) := x^2, f(x_, y_) := x + y )

In [34]:
Definition(f)

f(x_):=(x^2)
(f((x_),y_)):=(x + y)



In [35]:
ClearAll(f,a)

## Cpu time, memory, and tracing the evaluation


In [36]:
Timing((Range(10^6), Null ) )   # time a single expression

L"$$ [0.334739646,\text{Null}] $$"

`Time` toggles the timing of every expression entered. Memory allocated and the number of attempts to apply a user defined rule are also printed.

In [37]:
Time(True)  #  toggle timing all expressions. returns the previous value

L"$$ \text{False} $$"

In [38]:
Range(10^6);

  0.032541 seconds (1.00 M allocations: 22.947 MB, 35.58% gc time)
tryrule count: downvalue 0, upvalue 0


In [39]:
Time(False);

  0.000048 seconds (18 allocations: 1.563 KB)
tryrule count: downvalue 0, upvalue 0


In [40]:
Trace(True);  # Trace evaluation

1<< False


L"$$ \text{False} $$"

In [41]:
(a+b)*(a+b)

>>1 (a + b)*(a + b)
 >>2 a + b
 2<< a + b
 >>2 a + b
 2<< a + b
1<< (a + b)^(1 + 1)
>>1 (a + b)^(1 + 1)
 >>2 1 + 1
 2<< 2
1<< (a + b)^2
>>1 (a + b)^2
1<< (a + b)^2


L"$$  \left( a + b \right) ^{2} $$"

In [42]:
Trace(False);

>>1 Trace(False)


In [43]:
? LeafCount

```
LeafCount(expr)
```

gives the number of indivisible (`Part` can't be taken) elements in `expr`.

This amounts to counting all the `Heads` and all of the arguments that are not of type `Mxpr`, that is compound expressions.

A more accurate name is `NodeCount`.


Help( LeafCount)


 Attributes(LeafCount) = [Protected]


In [44]:
LeafCount(Expand((a+b)^3))

L"$$ 19 $$"

In [45]:
ByteCount(Expand((a+b)^3))

L"$$ 446 $$"

In [46]:
? Depth

```
Depth(expr)
```

gives the maximum number of indices required to specify any part of `expr`, plus `1`.


Help( Depth)


 Attributes(Depth) = [Protected]


In [47]:
Depth(Expand((a+b)^3))

L"$$ 4 $$"

In [48]:
FullForm(Expand((a+b)^3))  # Examine the tree

Plus(Power(a,3),Times(3,Power(a,2),b),Times(3,a,Power(b,2)),Power(b,3))

In [49]:
Expand((a+b)^3)[2,2,1]    # One of the deepest parts

L"$$ a $$"

## Calculations using expressions or functions

Here are a few ways to compute an integral

In [50]:
Integrate( (1+x^2)^(-1), x)

L"$$ \text{ArcTan} \!  \left( x \right)  $$"

In [51]:
expr = 1/(1+x^2);

In [52]:
Integrate(expr, x)

L"$$ \text{ArcTan} \!  \left( x \right)  $$"

In [53]:
f(x_) := 1/(1+x^2)

In [54]:
Integrate(f(y), y)

L"$$ \text{ArcTan} \!  \left( y \right)  $$"

In [55]:
g(x_) = expr # Note we do not use ":="

L"$$  \left( 1 + x^{2} \right) ^{-1} $$"

In [56]:
ClearAll(expr)  # We did not use SetDelay, so we can delete expr

In [57]:
Integrate(g(y),y)

L"$$ \text{ArcTan} \!  \left( y \right)  $$"

Note: Trying to use a compiled (Julia) function `h = :( x -> 1/(1+x^2))` will *not* work.

In [58]:
ClearAll(f,g,expr)

In [59]:
Integrate(f(y), y)  # The integral can no longer be reduced

L"$$ \int f \!  \left( y \right)  \,  \mathbb{d}y $$"

## Patterns and Matching

A `Blank` with no constraints matches everything

In [60]:
MatchQ(z,_)

L"$$ \text{True} $$"

In [61]:
Map(MatchQ(_), [1,"string", a+b, 1/3])  # MatchQ does Currying with the first argument

L"$$ [\text{True},\text{True},\text{True},\text{True}] $$"

`_head` is a `Blank` that only matches expressions with `Head` equal to `head`. 

In [62]:
FullForm(_Integer)  # underscore is shorthand for Blank

Blank(Integer)

In [63]:
MatchQ(1, _Integer)

L"$$ \text{True} $$"

Use Currying to define a predicate function

In [64]:
myintq = MatchQ(_Integer);

Not all rational numbers are integers

In [65]:
Map(myintq, Range(1/2,5,1/2))

L"$$ [\text{False},\text{True},\text{False},\text{True},\text{False},\text{True},\text{False},\text{True},\text{False},\text{True}] $$"

In [66]:
MatchQ(b^2, _^2)  # Match power with exponent equal to 2

L"$$ \text{True} $$"

In [67]:
MatchQ(b^3, _^_)   # Match any power

L"$$ \text{True} $$"

In [68]:
MatchQ((b+c)^3, _^_)

L"$$ \text{True} $$"

In [69]:
MatchQ(b^1, _^_)

L"$$ \text{False} $$"

This failed because `b^1` evaluates to `b`, which does not have the structure of a power

The pattern can be complex with blanks deep in an expression.

In [70]:
Map(MatchQ(f(x_^2)), [f(b^2), f(b^3), g(b^2)])

L"$$ [\text{True},\text{False},\text{False}] $$"

Specify a "function" `Head` that must match

In [71]:
Map( MatchQ(_gg), [gg(x+y), gg(x), g(x)])

L"$$ [\text{True},\text{True},\text{False}] $$"

Define a predicate for positive integers

In [72]:
m = MatchQ(_Integer:?(Positive));

In [73]:
Map(m, [1,100, 0, -1, 1.0, x])

L"$$ [\text{True},\text{True},\text{False},\text{False},\text{False},\text{False}] $$"

We can also put a condition on a `Pattern`. This matches pairs with the first element smaller than the second.

In [74]:
m = MatchQ(Condition([x_, y_], x < y));

In [75]:
[ m([2,1]), m([1,2]), m([1,2,3]), m(1)]

L"$$ [\text{False},\text{True},\text{False},\text{False}] $$"

`Pattern`s can include `Alternative`s.

In [76]:
m = MatchQ(_Integer | _String);

In [77]:
[m(1), m("zebra"), m(1.0)]

L"$$ [\text{True},\text{True},\text{False}] $$"

`Repeated(expr)` matches one or more occurences of `expr`.

In [78]:
MatchQ([a,a,a,b], [Repeated(a),b])

L"$$ \text{True} $$"

`RepeatedNull` matches zero or more occurences.

In [79]:
MatchQ([b], [RepeatedNull(a),b])

L"$$ \text{True} $$"

In [80]:
ClearAll(m)

## Replacing

`Rule`s are used for many things in Symata, including replacement. Replacement is a key ingredient in the implementation of functions.



When applied, this rule matches and does a replacement on any expression with `Head` `f` and a `List` of two elements as the sole argument.

In [81]:
f([x_,y_]) => p(x+y)

L"$$ f \!  \left( [x\text{_},y\text{_}] \right)  \Rightarrow p \!  \left( x + y \right)  $$"

In [82]:
expr = f([x+y,y]) + f(c) + g([a,b])

L"$$ f \!  \left( c \right)  + f \!  \left( [x + y,y] \right)  + g \!  \left( [a,b] \right)  $$"

In [83]:
ReplaceAll( expr, f([x_,y_]) => p(x+y))

L"$$ f \!  \left( c \right)  + g \!  \left( [a,b] \right)  + p \!  \left( x + 2 \ y \right)  $$"

There are several things to note here. 

- The pattern `x_` puts no restrictions on the match; any expression will match. The name of the pattern `x` only serves to identify it later during a replacement. 

- Here `x_` has matched `x+y`, but these two uses of `x` are not confused in the result. That is, in `x_`, the symbol `x` is a dummy variable. 

- The expression `f(c)` has a matching `Head`, but not matching arguments, so `f(c)` fails to match. Likewise, the expression `g([a,b])` has matching arguments, but not matching head.

- The expression `f([x+y,y])` matches, and the replacement is made in (a copy of) `expr`.  But, Symata alays evaluates expressions to a fixed point. So `y+y` is replaced by `2y`, and the terms in `expr` are rearranged into the canonical order.

Again, we have to be aware that matching is structural.

In [84]:
ReplaceAll([a/b, 1/b^2, 2/b^2] , b^n_ => d(n))

L"$$ [a \ d \!  \left( -1 \right) ,d \!  \left( -2 \right) ,2 \ d \!  \left( -2 \right) ] $$"

In [85]:
ClearAll(expr)

Named patterns that appear in more than one place must match the same expression.

In [86]:
ReplaceAll( [b,a,[a,b]] , [x_,y_,[x_,y_]] => 1 )  # This does not match

L"$$ [b,a,[a,b]] $$"

In [87]:
ReplaceAll( [a,b,[a,b]] , [x_,y_,[x_,y_]] => 1 )  # This does match

L"$$ 1 $$"

This example uses `Alternative`s.

In [88]:
 ReplaceAll( [a, b, c, d, a, b, b, b],  a | b => x)

L"$$ [x,x,c,d,x,x,x,x] $$"

The arguments of `Sequence` are spliced into expressions during evaluation.

In [89]:
[1,2,Sequence(a,b)]

L"$$ [1,2,a,b] $$"

An unmatched alternative is replaced by `Sequence()`. Upon evaluation to fixed point, this empty sequence is removed.

In [90]:
f(x_, x_ | y_String) := [x,y]

In [91]:
f(2,2)  # `y` does not match, so it is removed.

L"$$ [2] $$"

In [92]:
f(2,"cat")  # Here the second Alternative matches

L"$$ [2,\text{\"cat\"}] $$"

In [93]:
f(2,3)      # Here the Pattern fails to match.

L"$$ f \!  \left( 2,3 \right)  $$"

`Alternative`s, and `Pattern`s in general, can be explicit expressions, with no `Blank`s.

In [94]:
( h(a | b) := p, [h(a), h(b), h(c), h(d)] )

L"$$ [p,p,h \!  \left( c \right) ,h \!  \left( d \right) ] $$"

`ReplaceAll` replaces all matching subexpressions. We can also specify the levels. This matches at level 2 and deeper.

In [95]:
 Replace(1 + a + f(a) + g(f(a)), a => b, 2)

L"$$ 1 + b + f \!  \left( b \right)  + g \!  \left( f \!  \left( a \right)  \right)  $$"

This replaces only at level 2.

In [96]:
 Replace(1 + a + f(a) + g(f(a)), a => b, [2]) == 1 + a + f(b) + g(f(a))

L"$$ \text{True} $$"

`Rule` evaluates the right hand side once, when it is first evaluated.

In [97]:
ReplaceAll( [x,x,x,x,x],  x  => RandomReal() )

L"$$ [0.33275478361413446,0.33275478361413446,0.33275478361413446,0.33275478361413446,0.33275478361413446] $$"

`RuleDelayed` evaluates the right hand side every time it is applie

In [98]:
ReplaceAll( [x,x,x,x,x],  RuleDelayed(x ,RandomReal()))

L"$$ [0.7759833935241922,0.19749539941256833,0.8537537993824174,0.48638995862967493,0.7647745923391476] $$"

`Except` matches everything except expressions that match its argument. The following applies the replacement at level 1.

In [99]:
Replace([1, 7, "Hi", 3, Indeterminate], Except(_:?(NumericQ)) => 0, 1)

L"$$ [1,7,0,3,0] $$"

Each `Rule` in a `List` of `Rule`s is tried in turn. Matching stops after the first match. `ReplaceRepeated` continues applying `Rules` until the expression reaches a fixed point.

In [100]:
 ReplaceRepeated(x^2 + y^6 , List(x => 2 + a, a => 3))

L"$$ 25 + y^{6} $$"

Up to this point, we have used named patterns only with a single blank, for example `b_`. But, we may associate aname with any pattern expression, including a complex (compound) expression.

In [101]:
 ReplaceAll( b^c, a::(_^_) => g(a))

L"$$ g \!  \left( b^{c} \right)  $$"

`Patterns` are used to implement optional arguments.

In [102]:
f(x_, y_:3) := x + y

In [103]:
[f(a+b,z), f(a+b)]

L"$$ [a + b + z,3 + a + b] $$"

`Condition` may be used in definitions like this:

In [104]:
(ClearAll(f), f(x_) :=  Condition(x^2, x > 3))

In [105]:
[f(4),f(3)]

L"$$ [16,f \!  \left( 3 \right) ] $$"

We can match and replace with a `Pattern` with `Head` equal to `Plus`

In [106]:
ReplaceAll( z*y + b , x_ + y_ => x * y )

L"$$ b \ y \ z $$"

In [107]:
ReplaceAll( z*y + b +c , x_ + y_ => x * y)

L"$$ b + c + y \ z $$"

This failed because `Plus` with two terms does not match `Plus` with three terms. But, we actually *do* want this to match. Implementing associative-commuatative matching is a major goal of Symata. Anyone want to give it a try ?

In [108]:
ClearAll(f,h,a,b,x,y)

## Interface to compiled languages

Symata's host language is Julia, a high-performance, compiled language. It can be useful to call Juila code from Symata or to compile Symata code to Julia. Symata is also an open-source project, which means you can alter or add to it directly.

Define a compiled function to a built-in or user-defined Julia function like this

### Calling compiled functions

In [109]:
mylog = :( log );

In [110]:
mylog(2,2)

L"$$ 1.0 $$"

You can also easily write compiled code like this

In [111]:
f = :( (x,y)  ->  x^2 + y^2 );

In [112]:
f(3.0,4.0)

L"$$ 25.0 $$"

Note that we did not specify the data types. Is this really high-performance compiled code ? Yes it is. The function was compiled after we called it with two floating point numbers. If we call the function with two integers, a version (called a *method*) that is optimized for integers is compiled. A version optimized for an integer and a rational number or any combination of arguments can also be compiled.

In [113]:
[f(3,4), f(3, 1/2)]

L"$$ [25,\frac{37}{4}] $$"

We define two versions of the same function to see the difference in performance between compiled functions and functions defined via `Rule`s.

In [114]:
(a = Range(0.0,100.,.01), ccossq = :( x -> cos(x)^2 ), cossq(x_) := cos(x)^2);

In [115]:
Time(True);

We run each test twice because compilation time is included in the first run.

In [116]:
Map(cossq, a);

  1.916966 seconds (2.76 M allocations: 171.773 MB, 5.56% gc time)
tryrule count: downvalue 10001, upvalue 0


In [117]:
Map(cossq, a);

  1.325972 seconds (2.50 M allocations: 160.392 MB, 16.25% gc time)
tryrule count: downvalue 10001, upvalue 0


In [118]:
Map(ccossq, a);

  0.019304 seconds (33.25 k allocations: 777.481 KB)
tryrule count: downvalue 0, upvalue 0


In [119]:
Map(ccossq, a);

  0.001763 seconds (30.06 k allocations: 629.734 KB)
tryrule count: downvalue 0, upvalue 0


In [120]:
(Time(False), ClearAll(f,a,ccossq,cossq,mylog))

  0.002785 seconds (369 allocations: 21.453 KB)
tryrule count: downvalue 0, upvalue 0


The compiled function is about $500$ times faster in this example. Symata `Pattern`s can be compiled (automatically) as well, but this has been removed during a rewriting of the `Pattern` code that is still underway. With compiled `Pattern`s, the factor might be closer to $50$.

### Compiling Symata expressions

In this example we calculate an expression and compile it. The compiled code is as efficient hand-coded Julia.

In [121]:
expr = Integrate( x^2 * Exp(x)* Cos(x), x)

L"$$ \frac{- \  \mathbb{e} ^{x} \ \text{Cos} \!  \left( x \right) }{2} + \frac{ \mathbb{e} ^{x} \ x^{2} \ \text{Cos} \!  \left( x \right) }{2} + \frac{ \mathbb{e} ^{x} \ \text{Sin} \!  \left( x \right) }{2} + \frac{ \mathbb{e} ^{x} \ x^{2} \ \text{Sin} \!  \left( x \right) }{2} + - \  \mathbb{e} ^{x} \ x \ \text{Sin} \!  \left( x \right)  $$"

In [122]:
expr = Collect(expr, Exp(x))

L"$$  \mathbb{e} ^{x} \  \left( \frac{- \ \text{Cos} \!  \left( x \right) }{2} + \frac{x^{2} \ \text{Cos} \!  \left( x \right) }{2} + \frac{\text{Sin} \!  \left( x \right) }{2} + \frac{x^{2} \ \text{Sin} \!  \left( x \right) }{2} + - \ x \ \text{Sin} \!  \left( x \right)  \right)  $$"

In [123]:
( cexpr = Compile(expr),  a = Range(0.0,10.0,.01));

In [124]:
Timing((Map(cexpr, a), Null))

L"$$ [0.012629153,\text{Null}] $$"

In [125]:
Timing((Map(cexpr, a), Null))

L"$$ [0.000183746,\text{Null}] $$"

In [126]:
ClearAll(a,expr,cexpr)